In [ ]:
sm = snakemake

In [ ]:
import spherpro.bro as spb
import spherpro.db as db
import numpy as np
import pathlib
import loompy

### Aim: The goal is to find if there are markers which are significantly different with and without overexpression.

First it is investigated for the infected cells themselves. Approach:
- Fit a model to calculate:

    log(y_marker) ~ dist-to-rim + site (a random effect)
    
    
- Is is_overexpressing a singificant covariate?
- Calculate the residuals after applying this model, visualize the z-score 

In [ ]:
fn_config =  sm.input.fn_config

In [ ]:
from src.variables import Vars
from src.config import Conf

In [ ]:
bro = spb.get_bro(fn_config)

import spherpro.bromodules.helpers_vz as helpers_vz
imp.reload(helpers_vz)
hpr = helpers_vz.HelperVZ(bro)

In [ ]:
def censor_dat(x, q=99.99):
    x = np.copy(x)
    p = np.percentile(x,q=q)
    x[x> p] = p
    return x

def cur_logtransf(x):
    return np.log10(x+0.1)
transform = cur_logtransf

def cur_transf(x):
    x= censor_dat(x)
    x= cur_logtransf(x)
    return x

In [ ]:
V = Vars

In [ ]:
class Config(Conf):
    fn_constructs = pathlib.Path(sm.input.fn_constructs)
    REF_COND = 'ctrl'
    SUFFIX_NB = '-NB'
    FIL_FLAGPOS = 'is-flagpos'
    FIL_FLAGPOSNB = FIL_FLAGPOS+V.SUFFIX_FILNB
    FIL_GFPPOS = 'is-gfppos'
    FIL_GFPPOSNB = FIL_GFPPOS+V.SUFFIX_FILNB
    FIL_LM = 'modelfitcond_v1'
    FIL_LM_ONEFLAG = 'modelfitcondflag_v1'
    FIL_LM_ALLCTRL = 'modelfitcondallctrl_v1'
    FIL_LMS = [FIL_LM, FIL_LM_ONEFLAG]
    fn_lmdata = sm.output.fn_lmdata
C = Config

In [ ]:
q_obj = (bro.data.get_objectmeta_query()
             .filter(db.objects.object_type == 'cell')
            # .filter(db.objects.image_id == 101)
            .join(db.conditions, db.images.condition_id == db.conditions.condition_id)
             
            .add_columns(db.sites.site_id,
                        db.conditions.condition_id, db.conditions.plate_id,
                         db.conditions.bc_plate, db.conditions.sampleblock_id,
                        db.slides.slide_id, db.slideacs.slideac_id,
                        db.conditions.condition_name))

for fil_name in C.FIL_LMS:
    fil = (bro.session.query(db.object_filters)
               .join(db.object_filter_names)
               .filter(db.object_filter_names.object_filter_name == fil_name)).subquery()
    q_obj = (q_obj
         .join(fil, fil.c.object_id == db.objects.object_id)
         .add_columns(fil.c.filter_value.label(fil_name))
        )

In [ ]:
bro.data._read_pannel()
dat_pannel = bro.data.pannel

In [ ]:
fil = bro.filters.measurements.get_measmeta_filter_statements(channel_names=[tuple(dat_pannel.query(f'{V.COL_WORKING} == True')['metal'].values), 'object'],
                                                         measurement_names=['MeanIntensityComp', 'dist-rim'],
                                                             measurement_types=['Intensity', 'Location'],
                                                             stack_names=['FullStackFiltered', 'ObjectStack'])
q_meas = (bro.data.get_measmeta_query()
         .add_columns(db.ref_planes.channel_name,
                      db.stacks.stack_name,
                     db.ref_stacks.scale)
          .filter(fil)
         )

In [ ]:
%%time 
dat = bro.io.objmeasurements.get_measurements(q_meas=q_meas, q_obj=q_obj)
bro.io.objmeasurements.scale_anndata(dat)

In [ ]:
dat = dat[dat.obs[C.FIL_LM] > 0,:]

In [ ]:
%%time
dat.write_loom(C.fn_lmdata)
